In [1]:

# # Gene Regulatory Network Prediction of Yeast Network
# The task:
# Predict which transcription factors affects genes (For now this is non-tf)

In [2]:
# Import stuff
import pandas as pd
import numpy as np

In [7]:
# # Change directory to VSCode workspace root so that relative path loads work correctly. Turn this addition off with the DataScience.changeDirOnImportExport setting
# # ms-python.python added
# import os
# try:
# 	# os.chdir(os.path.join(os.getcwd(), '..'))
#     os.chdir(os.path.join(os.getcwd(), "./scripts-notebooks"))
#     print(os.getcwd())
# except Exception as e:
# 	print(e)

/Users/SamYang95/GRN_Inference_IY/scripts-notebooks


In [8]:
# Get expression data
ko_df = pd.read_csv('../dataset/yeast_networks/expression/KO.txt', sep='\t').set_index('Gene')
nv_df = pd.read_csv('../dataset/yeast_networks/expression/NatVar.txt', sep='\t').set_index('Gene')
stress_df = pd.read_csv('../dataset/yeast_networks/expression/Stress.txt', sep='\t').set_index('Name')

# Get Transcription Factors
ko_tf = pd.read_csv('../dataset/yeast_networks/expression/KO_TF_names.txt', sep='\t', header=None)
ko_tf.columns = ['TF']

nv_tf = pd.read_csv('../dataset/yeast_networks/expression/NatVar_TF_names.txt', sep='\t', header=None)
nv_tf.columns = ['TF']

stress_tf = pd.read_csv('../dataset/yeast_networks/expression/Stress_TF_names.txt', sep='\t', header=None)
stress_tf.columns = ['TF']

# Note that there are different gene list for each df, so we will use ko as test for now.

In [9]:
df = ko_df.merge(nv_df, how='inner', left_index=True, right_index=True)
df = df.merge(stress_df, how='inner', left_index=True, right_index=True)

tf = ko_tf.merge(nv_tf, how='inner', left_on='TF', right_on='TF')
tf = tf.merge(stress_tf, how='inner', left_on='TF', right_on='TF')

In [10]:
# Prepare dataset: set which are the predictors (TF), and which are the targets
from sklearn import model_selection
def prep_dataset(target_gene, tf_list, exp_df):
    '''
    Prepares training set and test set for target gene
    
    Args:
        - target_gene: target gene for the iteration (y)
        - exp_df: expression dataframe (already in pandas df format)
        - tf_list: transcription factors, which will be the predictors (X)
        
    Returns:
        - Training and Testing set to be used in model predictions
        - label for predictors, so we can subset this later
    '''
    # Get y (target) and predictor matrix (X)
    y = exp_df.loc[target_gene, :].values
    X = exp_df.loc[tf_list, :]
    
    if target_gene in tf_list.values:
        X = X.drop(index=target_gene)
    
    X_label = X.index # Predictor labels for X
    X = X.values.transpose()
    
    # Split 80:20 for test and train
    X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, test_size = 0.2)

    
    return X_train, X_test, y_train, y_test, X_label

In [11]:


from sklearn.linear_model import Lasso, LassoCV
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor

def grn_lasso(target_gene, tf_list, exp_df):
    '''
    GRN inference method using lasso regression
    
    Args:
        - target_gene: target gene for the iteration (y)
        - exp_df: expression dataframe (already in pandas df format)
        - tf_list: transcription factors, which will be the predictors (X)
        
    Returns:
        - Numpy array of type str, with list of non-zero weight predictors
    '''
    # Prep data
    X_train, X_test, y_train, y_test, X_label = prep_dataset(target_gene, tf_list, exp_df)
    
    # Use Lasso regression
    lasso_reg = LassoCV(alphas = [0.01, 0.1, 1, 10, 100], cv=5)
    lasso_reg.fit(X_train, y_train)
    
    # Get scores (R^2)
    train_score = lasso_reg.score(X_train, y_train) # Note: R^2 not very good, maybe use other methods
    test_score = lasso_reg.score(X_test, y_test)
    
    # Get weights of lasso, non zero weights are regulators
    predictors = X_label
    nonzero_filter = lasso_reg.coef_ != 0
    nonzero_preds = predictors[nonzero_filter]
    
    return nonzero_preds.values

def grn_regforest(target_gene, tf_list, exp_df):
    '''
    GRN inference method using regression forest. This method does not assume linearity of data.
    
    Args:
        - target_gene: target gene for the iteration (y)
        - exp_df: expression dataframe (already in pandas df format)
        - tf_list: transcription factors, which will be the predictors (X)
        
    Returns:
        - Numpy array of type str, with list of non-zero weight predictors
        
    '''
    # Prep data
    X_train, X_test, y_train, y_test, X_label = prep_dataset(target_gene, tf_list, exp_df)
    
    # Use regerssion tree
    forest_reg = RandomForestRegressor(n_estimators = 10, max_depth = 8, bootstrap = True, min_samples_leaf = 10, n_jobs=-1)
    forest_reg.fit(X_train, y_train)
    
    # Get Scores (R^2)
    train_score = forest_reg.score(X_train, y_train)
    test_score = forest_reg.score(X_test, y_test)
    
    # Get feature importance
    predictors = X_label
    nonzero_filter = forest_reg.feature_importances_ != 0
    nonzero_preds = predictors[nonzero_filter]
    
    return nonzero_preds.values



# Read in an expression file, and predict GRN using a method
def predict_grn(exp_df, tf_list, method, target_list=None, *args):
    '''
    Function that reads in expression level and predict its GRN using a specific method
    
    Args:
        - exp_df: expression dataframe (already in pandas df format), index is gene names, columns are sample / treatments
        - method: Method of inferring GRN (Example: grn_lasso)
        - *args: additional arguments for method
        - tf_list: transcription factors, which will be the predictors (X)
        
    Returns:
        - Set of predicted edges with format (regulator(space)target)
    '''
    
    pred_edges = set()

    if target_list is None:
        target_list = exp_df.index.values

    # for target in exp_df.index.values:
    for target in target_list:
        # Get regulators for every target gene

        pred_regs = method(target, tf_list, exp_df, *args)

        # Add the predicted regulators -> target edge to the predicted edges
        for reg in pred_regs:
            pred_edges.add(f'{reg}->{target}')
    
    return pred_edges

In [30]:
# Only train on a subset of the target genes to speed up the tuning process
num_target_genes = 10
indices = np.random.randint(df.shape[0], size=num_target_genes)
indices = [True if i in indices else False for i in range(df.shape[0])]
target_list = df.loc[indices].index.values

In [31]:
# Predict Edges
import time

time_0 = time.time()

regforest_edges = predict_grn(df, tf.loc[:, 'TF'], grn_regforest, target_list=target_list)
time_1 = time.time()
print("Time Training RegForest: ", time.strftime("%H:%M:%S", time.gmtime(time_1 - time_0)))

# lasso_edges = predict_grn(df, tf.loc[:, 'TF'], grn_lasso)
# time_2 = time.time()
# print("Time Training Lasso: ", time.strftime("%H:%M:%S", time.gmtime(time_2 - time_0)))

Time Training RegForest:  00:00:09


In [32]:

# Performance Metrics

def iou_score(gold_df, pred_grn_edges, target_list=None):
    '''
    Function to score predicted grn vs gold standard based on intersection over union.

    Score:
    For starters, we will use the simple score of intersection / union
    Intersection: Count of edges in both gold standard AND predicted grn
    Union: Count of gold standard edges + predicted grn edges - Intersection

    Intersection / Union is the score.

    Args:
        - gold_file: Path to gold standard file
        - pred_grn_edges: set of predicted grn edges

    Returns:
        - IOU score
    '''

    # Set of gold standard edges
    if target_list is not None:
        gold_df = gold_df[gold_df['Target'].isin(target_list)]


    gold_edge_set = set(gold_df.loc[:, 'Regulator'] + '->' + gold_df.loc[:, 'Target'])

    gold_regulators = set(gold_df.loc[:, 'Regulator'])
    pred_grn_edges = [edge for edge in pred_grn_edges if edge.split('->')[0] in gold_regulators]

    # Get Intersection and Union
    intersection = gold_edge_set.intersection(pred_grn_edges)
    union = gold_edge_set.union(pred_grn_edges)

    # Get score: Intersection / Union
    iou_score = len(intersection) / len(union)
    print(f'Union edges count: {len(union)}')
    print(f'Intersection edges count: {len(intersection)}')
        
    return iou_score

In [33]:
import os
ko_gold_df = pd.read_csv("../dataset/yeast_networks/gold/MacIsaac2.KO.txt", sep = '\t', header = None, names = ['Regulator', 'Target'])
nv_gold_df = pd.read_csv("../dataset/yeast_networks/gold/MacIsaac2.NatVar.txt", sep = '\t', header = None, names = ['Regulator', 'Target'])
stress_gold_df = pd.read_csv("../dataset/yeast_networks/gold/MacIsaac2.Stress.txt", sep = '\t', header = None, names = ['Regulator', 'Target'])

gold_df = ko_gold_df.merge(nv_gold_df, how='outer', left_on=['Regulator', 'Target'], right_on=['Regulator', 'Target'])
gold_df = gold_df.merge(stress_gold_df, how='outer', left_on=['Regulator', 'Target'], right_on=['Regulator', 'Target'])

In [34]:
FinallyItIsDone()

In [35]:
# lasso_score = iou_score(gold_df, lasso_edges)
regforest_score = iou_score(gold_df, regforest_edges, target_list=target_list)

Union edges count: 19
Intersection edges count: 2


In [ ]:


print(f'KO Lasso score: {lasso_score}') # Much better than last time in try 1
print(f'KO Regforest score: {regforest_score}') 

In [28]:
# Sound alert after code is finished!
from IPython.display import Audio, display
def FinallyItIsDone():
  display(Audio(url='https://sound.peal.io/ps/audios/000/000/537/original/woo_vu_luvub_dub_dub.mp3', autoplay=True))